In [1]:
################################################################ TO CHANGE ################################################################
# Stolen from Sneha
# FILE PATHS AND FIELDS
path_json = "../comp767_papers_sample.jsonl"  #3154 papers
fields = ["title", "abstract", "authors"] # fields to include in training
# TRAINING PARAM
num_topics = 10 # truthfully we want to see 13 topics
chunksize = 2000 # how many docs are processed at a time set to 2000 as default
passes = 20 # how often the model is trained on all the docs set to 20 as default
iterations = 400 # how often do we iterate over each doc set to 400 as default
eval_every = None  # Don't evaluate model perplexity, takes too much time.
################################################################ TO CHANGE ################################################################


import json #
import nltk # for preprocessing
nltk.download('wordnet')

from nltk.tokenize import RegexpTokenizer # for tokenization
from nltk.stem.wordnet import WordNetLemmatizer # for lemmatizing
from gensim.corpora import Dictionary # to construct dictionary
from gensim.models import LdaModel # to make LDA model
from pprint import pprint # print output in a readable way
from nltk.util import ngrams

import pyLDAvis.gensim

with open(path_json) as fp:
    papers = [json.loads(line) for line in fp.readlines()]

    

/Users/baddie/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/Users/baddie/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and

# Preprocessing functions 

## Preprocessing Methods - Abstract
1. n-grams
1. remove articles + small words

## Preprocessing Methods - Authors
1. n-grams
1. Throwing out authors all together


In [2]:
def ident(z,*args):
    '''dummy identity function'''
    if (type(z) is not list):
        return z
    else:
        return ' '.join(z)
    
def author_iden(z,*args):
    return z 

def add_ngrams(inpt_sentence, n=[1]):
    
    if inpt_sentence is not None:
        
        out=inpt_sentence
        
        for i in n:
            
            grams=ngrams(inpt_sentence, i)
            
            out.extend(['_'.join(x) for x in grams])
        
        return ' '.join(out)
    return ''

def author_ngram(input_list, *args):
    return [x.replace(' ', '_').lower() for x in input_list]

def destroy_param(z,*args):
    return []

def preprocess_data(all_docs, min_word_len=2,
                    title_pp=ident,arg_title=None,
                    abstract_pp=ident, arg_abstract=None,
                    author_pp=author_iden, arg_author=None):
    ret_ar=[]
    
    tokenizer = RegexpTokenizer(r'\w+')
    lemmatizer = WordNetLemmatizer()
    for doc in all_docs:
        
        #title
        title= ' '.join([lemmatizer.lemmatize (x) for x in doc['title'].split(' ')])
        
        #abstract
        abstract= [lemmatizer.lemmatize (x) for x in str(doc['abstract']).split(' ')] #list
        
        abstract = [x for x in abstract if len(x)>min_word_len]
        
        # concatenate all strings 
        representation = title_pp(title,arg_title) + ' \n '+ abstract_pp(abstract,arg_abstract).lower()
        
        # get rid of punctuation & tokenize
        representation=tokenizer.tokenize(representation.lower()) + author_pp(doc['authors'],arg_author)
        
        # take out numbers (but not numbers within words)
        representation = [token for token in representation if not token.isnumeric()]

        # take out words that are at least 3 characters long character
        representation = [token for token in representation if len(token) > min_word_len] 

        # channge code here to not lemmatize ngrams
        #representation = [lemmatizer.lemmatize(token) for token in representation]

        representation=[x.strip('_') for x in representation]
        ret_ar.append(representation)
    
    return ret_ar


### Experiment 1 : 
Author ngram  + standard abstract

In [ ]:
#corpus preprocessing
full_data=preprocess_data(papers,
                          author_pp= author_ngram )

#constructs word to ID mapping 
dictionary = Dictionary(full_data)

# filters out words that occur less than 20 times or are in more than 50% of docs
dictionary.filter_extremes(no_below=20, no_above=0.5)

# transform to vectorized form to put in model
corpus = [dictionary.doc2bow(doc) for doc in full_data]

# Finds how many unique tokens we've found and how many docs we have
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

# index to word dictionary
temp = dictionary[0] 
id2word = dictionary.id2token

for num_topics in range(10,16):
    model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )

    # sum of topic coherences of all topics, divided by the number of topics
    avg_topic_coherence = sum([t[1] for t in model.top_topics(corpus)]) / num_topics 
    print('Average topic coherence: %.4f.' % avg_topic_coherence)

    param_changes="author_ngram_standard_abstract_"+str(num_topics)+"_topics_"+str(round(avg_topic_coherence,4))+'_atc'
    visualisation = pyLDAvis.gensim.prepare(model, corpus, dictionary)
    full_output_path =  "./visualization/LDA_Visualization_" + param_changes + ".html"
    pprint(model.print_topics())
    pyLDAvis.save_html(visualisation, full_output_path)
    
    model.save("./models/LDA_" + param_changes + ".model")

### Experiment 2 : 
Author ngram  + Abstract ngram

In [3]:
############################### BIGRAMS ###############################

#corpus preprocessing
full_data=preprocess_data(papers,
                          author_pp= author_ngram,
                          abstract_pp=add_ngrams,arg_abstract=[2])

#constructs word to ID mapping 
dictionary = Dictionary(full_data)

# filters out words that occur less than 20 times or are in more than 50% of docs
dictionary.filter_extremes(no_below=20, no_above=0.5)

# transform to vectorized form to put in model
corpus = [dictionary.doc2bow(doc) for doc in full_data]

# Finds how many unique tokens we've found and how many docs we have
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

# index to word dictionary
temp = dictionary[0] 
id2word = dictionary.id2token

for num_topics in range(10,16):
    model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )

    # sum of topic coherences of all topics, divided by the number of topics
    avg_topic_coherence = sum([t[1] for t in model.top_topics(corpus)]) / num_topics 
    print('Average topic coherence: %.4f.' % avg_topic_coherence)
    param_changes="author_ngram_abstract_bigram_"+str(num_topics)+"_topics_"+str(round(avg_topic_coherence,4))+'_atc'
    visualisation = pyLDAvis.gensim.prepare(model, corpus, dictionary)
    full_output_path =  "./visualization/LDA_Visualization_" + param_changes + ".html"
    pprint(model.print_topics())
    pyLDAvis.save_html(visualisation, full_output_path)
    model.save("./models/LDA_" + param_changes + ".model")
    
############################### TRIGRAMS ###############################

#corpus preprocessing
full_data=preprocess_data(papers,
                          author_pp= author_ngram,
                          abstract_pp=add_ngrams,arg_abstract=[3])

#constructs word to ID mapping 
dictionary = Dictionary(full_data)

# filters out words that occur less than 20 times or are in more than 50% of docs
dictionary.filter_extremes(no_below=20, no_above=0.5)

# transform to vectorized form to put in model
corpus = [dictionary.doc2bow(doc) for doc in full_data]

# Finds how many unique tokens we've found and how many docs we have
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

# index to word dictionary
temp = dictionary[0] 
id2word = dictionary.id2token

for num_topics in range(10,16):
    model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )

    # sum of topic coherences of all topics, divided by the number of topics
    avg_topic_coherence = sum([t[1] for t in model.top_topics(corpus)]) / num_topics 
    print('Average topic coherence: %.4f.' % avg_topic_coherence)
    param_changes="author_ngram_abstract_trigram_"+str(num_topics)+"_topics_"+str(round(avg_topic_coherence,4))+'_atc'
    visualisation = pyLDAvis.gensim.prepare(model, corpus, dictionary)
    full_output_path =  "./visualization/LDA_Visualization_" + param_changes + ".html"
    pprint(model.print_topics())
    pyLDAvis.save_html(visualisation, full_output_path)
    model.save("./models/LDA_" + param_changes + ".model")
    
############################### BI-TRIGRAMS ###############################

#corpus preprocessing
full_data=preprocess_data(papers,
                          author_pp= author_ngram,
                          abstract_pp=add_ngrams,arg_abstract=[2,3])

#constructs word to ID mapping 
dictionary = Dictionary(full_data)

# filters out words that occur less than 20 times or are in more than 50% of docs
dictionary.filter_extremes(no_below=20, no_above=0.5)

# transform to vectorized form to put in model
corpus = [dictionary.doc2bow(doc) for doc in full_data]

# Finds how many unique tokens we've found and how many docs we have
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

# index to word dictionary
temp = dictionary[0] 
id2word = dictionary.id2token

for num_topics in range(10,16):
    model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )

    # sum of topic coherences of all topics, divided by the number of topics
    avg_topic_coherence = sum([t[1] for t in model.top_topics(corpus)]) / num_topics 
    print('Average topic coherence: %.4f.' % avg_topic_coherence)
    param_changes="author_ngram_abstract_bi-trigram_"+str(num_topics)+"_topics_"+str(round(avg_topic_coherence,4))+'_atc'
    visualisation = pyLDAvis.gensim.prepare(model, corpus, dictionary)
    full_output_path =  "./visualization/LDA_Visualization_" + param_changes + ".html"
    pprint(model.print_topics())
    pyLDAvis.save_html(visualisation, full_output_path)
    model.save("./models/LDA_" + param_changes + ".model")

Number of unique tokens: 3546
Number of documents: 3154
Average topic coherence: -1.7896.
[(0,
  '0.017*"research" + 0.008*"study" + 0.007*"social" + 0.007*"these" + '
  '0.007*"science" + 0.006*"have" + 0.006*"their" + 0.006*"how" + '
  '0.006*"policy" + 0.005*"analysis"'),
 (1,
  '0.021*"model" + 0.014*"temperature" + 0.013*"skill" + 0.012*"climate" + '
  '0.011*"over" + 0.011*"forecast" + 0.010*"sst" + 0.010*"seasonal" + '
  '0.010*"precipitation" + 0.009*"surface"'),
 (2,
  '0.022*"news" + 0.021*"medium" + 0.018*"social" + 0.015*"content" + '
  '0.015*"analysis" + 0.011*"media" + 0.011*"public" + 0.010*"coverage" + '
  '0.010*"communication" + 0.010*"online"'),
 (3,
  '0.018*"energy" + 0.014*"power" + 0.011*"cost" + 0.010*"emission" + '
  '0.008*"environmental" + 0.008*"water" + 0.007*"system" + 0.007*"cycle" + '
  '0.006*"gas" + 0.006*"carbon"'),
 (4,
  '0.023*"disclosure" + 0.017*"carbon" + 0.013*"firm" + 0.011*"environmental" '
  '+ 0.011*"company" + 0.010*"information" + 0.009*

Average topic coherence: -2.1977.
[(0,
  '0.021*"model" + 0.015*"network" + 0.011*"based" + 0.010*"method" + '
  '0.010*"problem" + 0.009*"which" + 0.009*"multi" + 0.009*"neural" + '
  '0.008*"time" + 0.007*"system"'),
 (1,
  '0.028*"disclosure" + 0.018*"carbon" + 0.016*"firm" + 0.013*"environmental" '
  '+ 0.013*"company" + 0.011*"corporate" + 0.011*"information" + '
  '0.009*"reporting" + 0.009*"study" + 0.008*"firms"'),
 (2,
  '0.025*"energy" + 0.010*"cost" + 0.009*"system" + 0.008*"can" + '
  '0.008*"demand" + 0.007*"use" + 0.006*"consumption" + 0.006*"building" + '
  '0.006*"urban" + 0.006*"smart"'),
 (3,
  '0.065*"climate" + 0.059*"change" + 0.031*"climate_change" + '
  '0.016*"education" + 0.010*"carbon" + 0.010*"global" + 0.009*"environmental" '
  '+ 0.006*"students" + 0.006*"their" + 0.006*"impact"'),
 (4,
  '0.013*"coverage" + 0.012*"medium" + 0.010*"more" + 0.010*"over" + '
  '0.009*"find" + 0.009*"time" + 0.007*"negative" + 0.007*"public" + '
  '0.007*"show" + 0.006*"our"')

Average topic coherence: -2.0701.
[(0,
  '0.021*"news" + 0.019*"medium" + 0.012*"coverage" + 0.011*"public" + '
  '0.011*"social" + 0.011*"media" + 0.010*"however" + 0.008*"study" + '
  '0.007*"their" + 0.007*"content"'),
 (1,
  '0.021*"political" + 0.018*"party" + 0.017*"policy" + 0.010*"their" + '
  '0.009*"government" + 0.007*"how" + 0.007*"state" + 0.007*"which" + '
  '0.007*"find" + 0.006*"more"'),
 (2,
  '0.031*"text" + 0.024*"analysis" + 0.014*"method" + 0.013*"data" + '
  '0.010*"automated" + 0.010*"content" + 0.010*"coding" + 0.009*"political" + '
  '0.009*"our" + 0.009*"using"'),
 (3,
  '0.009*"have" + 0.007*"however" + 0.007*"but" + 0.007*"study" + '
  '0.007*"product" + 0.006*"more" + 0.006*"emission" + 0.006*"not" + '
  '0.006*"development" + 0.006*"carbon"'),
 (4,
  '0.027*"network" + 0.026*"time" + 0.017*"prediction" + 0.013*"neural" + '
  '0.013*"data" + 0.013*"based" + 0.013*"real" + 0.012*"using" + '
  '0.011*"algorithm" + 0.011*"disruption"'),
 (5,
  '0.023*"model" +

Number of unique tokens: 3861
Number of documents: 3154
Average topic coherence: -1.9576.
[(0,
  '0.029*"text" + 0.026*"analysis" + 0.018*"topic" + 0.014*"content" + '
  '0.009*"topics" + 0.009*"language" + 0.008*"texts" + 0.008*"data" + '
  '0.007*"political" + 0.007*"word"'),
 (1,
  '0.012*"however" + 0.010*"party" + 0.010*"first" + 0.009*"second" + '
  '0.009*"policy" + 0.008*"political" + 0.008*"find" + 0.007*"government" + '
  '0.007*"public" + 0.007*"news"'),
 (2,
  '0.033*"data" + 0.020*"however" + 0.015*"learning" + 0.010*"model" + '
  '0.009*"our" + 0.009*"machine" + 0.009*"based" + 0.008*"models" + '
  '0.008*"which" + 0.008*"method"'),
 (3,
  '0.032*"change" + 0.022*"climate" + 0.015*"climate_change" + 0.010*"however" '
  '+ 0.009*"education" + 0.007*"development" + 0.006*"environmental" + '
  '0.006*"study" + 0.006*"health" + 0.006*"students"'),
 (4,
  '0.032*"model" + 0.018*"term" + 0.017*"long" + 0.010*"models" + 0.008*"self" '
  '+ 0.008*"short" + 0.008*"disruption" + 0.

Average topic coherence: -2.3512.
[(0,
  '0.022*"model" + 0.015*"sst" + 0.015*"temperature" + 0.011*"skill" + '
  '0.011*"climate" + 0.011*"enso" + 0.010*"variability" + '
  '0.010*"precipitation" + 0.010*"region" + 0.009*"ocean"'),
 (1,
  '0.019*"design" + 0.017*"methodology" + 0.015*"chapter" + 0.014*"study" + '
  '0.014*"research" + 0.013*"approach" + 0.012*"findings" + 0.009*"thus" + '
  '0.009*"project" + 0.008*"field"'),
 (2,
  '0.033*"however" + 0.011*"party" + 0.011*"government" + 0.009*"political" + '
  '0.008*"which" + 0.008*"but" + 0.008*"states" + 0.008*"public" + '
  '0.007*"their" + 0.006*"parties"'),
 (3,
  '0.028*"disclosure" + 0.018*"carbon" + 0.015*"firm" + 0.014*"ghg" + '
  '0.013*"firms" + 0.013*"companies" + 0.011*"company" + 0.010*"information" + '
  '0.010*"csr" + 0.010*"cdp"'),
 (4,
  '0.031*"data" + 0.021*"learning" + 0.014*"however" + 0.013*"based" + '
  '0.012*"machine" + 0.009*"our" + 0.009*"algorithm" + 0.009*"paper" + '
  '0.008*"this_paper" + 0.008*"model

### Experiment 3 : 
No author  + Abstract ngram


In [4]:
############################### BIGRAMS ###############################
#corpus preprocessing
full_data=preprocess_data(papers,
                          author_pp= destroy_param,
                          abstract_pp=add_ngrams,arg_abstract=[2])

#constructs word to ID mapping 
dictionary = Dictionary(full_data)

# filters out words that occur less than 20 times or are in more than 50% of docs
dictionary.filter_extremes(no_below=20, no_above=0.5)

# transform to vectorized form to put in model
corpus = [dictionary.doc2bow(doc) for doc in full_data]

# Finds how many unique tokens we've found and how many docs we have
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

# index to word dictionary
temp = dictionary[0] 
id2word = dictionary.id2token

for num_topics in range(10,16):
    model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )

    # sum of topic coherences of all topics, divided by the number of topics
    avg_topic_coherence = sum([t[1] for t in model.top_topics(corpus)]) / num_topics 
    print('Average topic coherence: %.4f.' % avg_topic_coherence)
    param_changes="author_none_abstract_bigram_"+str(num_topics)+"_topics_"+str(round(avg_topic_coherence,4))+'_atc'
    visualisation = pyLDAvis.gensim.prepare(model, corpus, dictionary)
    full_output_path =  "./visualization/LDA_Visualization_" + param_changes + ".html"
    pprint(model.print_topics())
    pyLDAvis.save_html(visualisation, full_output_path)
    model.save("./models/LDA_" + param_changes + ".model")


############################### TRIGRAMS ###############################
#corpus preprocessing
full_data=preprocess_data(papers,
                          author_pp= destroy_param,
                          abstract_pp=add_ngrams,arg_abstract=[3])

#constructs word to ID mapping 
dictionary = Dictionary(full_data)

# filters out words that occur less than 20 times or are in more than 50% of docs
dictionary.filter_extremes(no_below=20, no_above=0.5)

# transform to vectorized form to put in model
corpus = [dictionary.doc2bow(doc) for doc in full_data]

# Finds how many unique tokens we've found and how many docs we have
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

# index to word dictionary
temp = dictionary[0] 
id2word = dictionary.id2token

for num_topics in range(10,16):
    model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )

    # sum of topic coherences of all topics, divided by the number of topics
    avg_topic_coherence = sum([t[1] for t in model.top_topics(corpus)]) / num_topics 
    print('Average topic coherence: %.4f.' % avg_topic_coherence)
    param_changes="author_none_abstract_trigram_"+str(num_topics)+"_topics_"+str(round(avg_topic_coherence,4))+'_atc'
    visualisation = pyLDAvis.gensim.prepare(model, corpus, dictionary)
    full_output_path =  "./visualization/LDA_Visualization_" + param_changes + ".html"
    pprint(model.print_topics())
    pyLDAvis.save_html(visualisation, full_output_path)
    model.save("./models/LDA_" + param_changes + ".model")
    
############################### BI-TRIGRAMS ###############################
#corpus preprocessing
full_data=preprocess_data(papers,
                          author_pp= destroy_param,
                          abstract_pp=add_ngrams,arg_abstract=[2,3])

#constructs word to ID mapping 
dictionary = Dictionary(full_data)

# filters out words that occur less than 20 times or are in more than 50% of docs
dictionary.filter_extremes(no_below=20, no_above=0.5)

# transform to vectorized form to put in model
corpus = [dictionary.doc2bow(doc) for doc in full_data]

# Finds how many unique tokens we've found and how many docs we have
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

# index to word dictionary
temp = dictionary[0] 
id2word = dictionary.id2token

for num_topics in range(10,16):
    model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )

    # sum of topic coherences of all topics, divided by the number of topics
    avg_topic_coherence = sum([t[1] for t in model.top_topics(corpus)]) / num_topics 
    print('Average topic coherence: %.4f.' % avg_topic_coherence)
    param_changes="author_none_abstract_bi-trigram_"+str(num_topics)+"_topics_"+str(round(avg_topic_coherence,4))+'_atc'
    visualisation = pyLDAvis.gensim.prepare(model, corpus, dictionary)
    full_output_path =  "./visualization/LDA_Visualization_" + param_changes + ".html"
    pprint(model.print_topics())
    pyLDAvis.save_html(visualisation, full_output_path)
    model.save("./models/LDA_" + param_changes + ".model")

Number of unique tokens: 3545
Number of documents: 3154
Average topic coherence: -2.1388.
[(0,
  '0.040*"energy" + 0.030*"none" + 0.018*"city" + 0.017*"states" + '
  '0.017*"consumption" + 0.016*"urban" + 0.016*"building" + 0.015*"die" + '
  '0.014*"united" + 0.013*"use"'),
 (1,
  '0.021*"data" + 0.018*"social" + 0.016*"research" + 0.011*"study" + '
  '0.009*"analysis" + 0.006*"medium" + 0.006*"public" + 0.006*"their" + '
  '0.006*"science" + 0.006*"online"'),
 (2,
  '0.035*"data" + 0.028*"learning" + 0.024*"model" + 0.019*"machine" + '
  '0.015*"based" + 0.013*"method" + 0.010*"our" + 0.009*"using" + '
  '0.009*"machine_learning" + 0.008*"algorithm"'),
 (3,
  '0.024*"text" + 0.019*"analysis" + 0.019*"topic" + 0.010*"news" + '
  '0.010*"content" + 0.008*"method" + 0.007*"political" + 0.006*"model" + '
  '0.006*"approach" + 0.006*"our"'),
 (4,
  '0.029*"network" + 0.022*"time" + 0.014*"neural" + 0.011*"disruption" + '
  '0.011*"real" + 0.009*"networks" + 0.009*"which" + 0.009*"performan

Average topic coherence: -2.3330.
[(0,
  '0.028*"data" + 0.015*"research" + 0.009*"method" + 0.008*"analysis" + '
  '0.008*"topic" + 0.008*"these" + 0.008*"social" + 0.007*"can" + 0.007*"our" '
  '+ 0.007*"text"'),
 (1,
  '0.043*"change" + 0.042*"climate" + 0.021*"climate_change" + '
  '0.013*"education" + 0.007*"study" + 0.007*"their" + 0.006*"environmental" + '
  '0.006*"development" + 0.006*"global" + 0.006*"research"'),
 (2,
  '0.027*"disclosure" + 0.016*"firm" + 0.014*"carbon" + 0.012*"company" + '
  '0.011*"environmental" + 0.011*"information" + 0.011*"corporate" + '
  '0.009*"study" + 0.009*"reporting" + 0.008*"financial"'),
 (3,
  '0.052*"party" + 0.018*"speech" + 0.015*"parties" + 0.012*"show" + '
  '0.011*"enso" + 0.011*"position" + 0.010*"electoral" + 0.009*"government" + '
  '0.009*"their" + 0.009*"legislative"'),
 (4,
  '0.030*"text" + 0.027*"analysis" + 0.016*"content" + 0.014*"topic" + '
  '0.012*"method" + 0.010*"word" + 0.010*"sentiment" + 0.010*"automated" + '
  '0.00

Average topic coherence: -2.1336.
[(0,
  '0.013*"disruption" + 0.012*"candidate" + 0.012*"edu" + 0.011*"university" + '
  '0.011*"crisis" + 0.010*"data" + 0.010*"time" + 0.008*"non" + 0.008*"have" + '
  '0.007*"candidates"'),
 (1,
  '0.017*"network" + 0.015*"city" + 0.014*"urban" + 0.012*"self" + '
  '0.011*"demand" + 0.010*"service" + 0.010*"world" + 0.009*"finally" + '
  '0.009*"term" + 0.009*"scale"'),
 (2,
  '0.042*"energy" + 0.029*"power" + 0.019*"system" + 0.016*"cost" + '
  '0.013*"cycle" + 0.012*"emission" + 0.011*"systems" + 0.011*"gas" + '
  '0.011*"plant" + 0.010*"water"'),
 (3,
  '0.031*"model" + 0.017*"data" + 0.016*"learning" + 0.014*"method" + '
  '0.013*"network" + 0.013*"time" + 0.013*"algorithm" + 0.012*"machine" + '
  '0.012*"based" + 0.011*"using"'),
 (4,
  '0.034*"news" + 0.024*"medium" + 0.020*"coverage" + 0.014*"public" + '
  '0.012*"article" + 0.011*"media" + 0.011*"content" + 0.010*"government" + '
  '0.010*"policy" + 0.009*"newspaper"'),
 (5,
  '0.017*"risk" +

Number of unique tokens: 3860
Number of documents: 3154
Average topic coherence: -1.9995.
[(0,
  '0.017*"disclosure" + 0.012*"addition" + 0.009*"company" + 0.009*"first" + '
  '0.009*"second" + 0.008*"companies" + 0.008*"study" + 0.008*"environmental" '
  '+ 0.007*"firm" + 0.007*"carbon"'),
 (1,
  '0.018*"specifically" + 0.013*"firms" + 0.012*"reports" + 0.012*"our" + '
  '0.012*"cdp" + 0.011*"furthermore" + 0.010*"moreover" + 0.010*"however" + '
  '0.010*"information" + 0.009*"study"'),
 (2,
  '0.023*"policy" + 0.019*"however" + 0.017*"find" + 0.014*"states" + '
  '0.013*"public" + 0.011*"news" + 0.008*"state" + 0.008*"yet" + 0.008*"china" '
  '+ 0.007*"conflict"'),
 (3,
  '0.016*"party" + 0.014*"topic" + 0.011*"political" + 0.010*"however" + '
  '0.009*"topics" + 0.008*"parties" + 0.008*"time" + 0.008*"which" + '
  '0.007*"first" + 0.007*"lda"'),
 (4,
  '0.024*"energy" + 0.013*"system" + 0.011*"power" + 0.011*"cost" + '
  '0.010*"systems" + 0.010*"emissions" + 0.009*"environmental" +

Average topic coherence: -2.7195.
[(0,
  '0.022*"carbon" + 0.016*"emissions" + 0.014*"ghg" + 0.013*"emission" + '
  '0.011*"energy" + 0.010*"cdp" + 0.009*"non" + 0.008*"management" + '
  '0.008*"cost" + 0.007*"gas"'),
 (1,
  '0.052*"change" + 0.035*"climate" + 0.023*"climate_change" + '
  '0.015*"education" + 0.009*"students" + 0.008*"environmental" + '
  '0.008*"world" + 0.006*"knowledge" + 0.006*"teachers" + 0.006*"development"'),
 (2,
  '0.023*"research" + 0.020*"moreover" + 0.012*"analysis" + 0.012*"social" + '
  '0.011*"article" + 0.010*"data" + 0.009*"content" + 0.008*"field" + '
  '0.008*"which" + 0.008*"science"'),
 (3,
  '0.075*"first" + 0.057*"second" + 0.049*"finally" + 0.018*"term" + '
  '0.018*"third" + 0.015*"however" + 0.011*"hence" + 0.010*"then" + '
  '0.009*"find" + 0.008*"opposition"'),
 (4,
  '0.033*"edu" + 0.028*"university" + 0.024*"science" + 0.022*"data" + '
  '0.016*"com" + 0.014*"dimensions" + 0.013*"department" + 0.012*"finally" + '
  '0.011*"secondly" + 0.01

The number of unique tokens not really changing may mean that the authors are not super important

### Experiment 4 : 
No author  + standard Abstract


In [ ]:
#corpus preprocessing
full_data=preprocess_data(papers,
                          author_pp= destroy_param )

#constructs word to ID mapping 
dictionary = Dictionary(full_data)

# filters out words that occur less than 20 times or are in more than 50% of docs
dictionary.filter_extremes(no_below=20, no_above=0.5)

# transform to vectorized form to put in model
corpus = [dictionary.doc2bow(doc) for doc in full_data]

# Finds how many unique tokens we've found and how many docs we have
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

# index to word dictionary
temp = dictionary[0] 
id2word = dictionary.id2token

for num_topics in range(10,16):
    model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )

    # sum of topic coherences of all topics, divided by the number of topics
    avg_topic_coherence = sum([t[1] for t in model.top_topics(corpus)]) / num_topics 
    print('Average topic coherence: %.4f.' % avg_topic_coherence)
    param_changes="author_none_abstract_standard_"+str(num_topics)+"_topics_"+str(round(avg_topic_coherence,4))+'_atc'
    visualisation = pyLDAvis.gensim.prepare(model, corpus, dictionary)
    full_output_path =  "./visualization/LDA_Visualization_" + param_changes + ".html"
    pprint(model.print_topics())
    pyLDAvis.save_html(visualisation, full_output_path)
    model.save("./models/LDA_" + param_changes + ".model")

In [ ]:
dir(model)

In [40]:
model.num_topics

15